<a href="https://colab.research.google.com/github/NagaoTadashi/Ledge-recommend/blob/main/baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [151]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [152]:
!wget -nc --no-check-certificate https://files.grouplens.org/datasets/movielens/ml-10m.zip -P ../data
!unzip -n ../data/ml-10m.zip -d ../data/

File ‘../data/ml-10m.zip’ already there; not retrieving.

Archive:  ../data/ml-10m.zip


映画情報

In [153]:
m_cols = ['movie_id', 'title', 'genre']
df_movies = pd.read_csv('../data/ml-10M100K/movies.dat', names=m_cols, sep='::' , encoding='latin-1', engine='python')

In [154]:
df_movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [155]:
print("映画の作品数：{}".format(len(df_movies["movie_id"].unique())))

映画の作品数：10681


In [156]:
df_movies["genre"] = df_movies["genre"].str.split("|")

In [158]:
df_movies.explode("genre")

,movie_id,title,genre
0,1,Toy Story (1995),Adventure
0,1,Toy Story (1995),Animation
0,1,Toy Story (1995),Children
0,1,Toy Story (1995),Comedy
0,1,Toy Story (1995),Fantasy
...,...,...,...
10678,65126,Choke (2008),Comedy
10678,65126,Choke (2008),Drama
10679,65130,Revolutionary Road (2008),Drama
10679,65130,Revolutionary Road (2008),Romance


In [135]:
_df = df_movies.copy()

_df = (
    _df.explode('genre')
    .pivot_table(index='title', 
                 columns='genre', 
                 values='movie_id', 
                 aggfunc='count')
                 .fillna(0)
                 .astype(int)
)

genre_mat = np.array(_df)

norm = np.matrix(np.linalg.norm(genre_mat, axis=1))
similarity = np.array(
    np.dot(genre_mat, genre_mat.T)/np.dot(norm.T, norm)
)

df_sim = pd.DataFrame(similarity, index=_df.index, columns=_df.index)

In [137]:
df_sim.head()

title,"""Great Performances"" Cats (1998)",'Round Midnight (1986),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),*batteries not included (1987),...All the Marbles (a.k.a. The California Dolls) (1981),...And God Created Woman (Et Dieu... crÃ©a la femme) (1956),...And God Spoke (1993),...And Justice for All (1979),...,Zorba the Greek (Alexis Zorbas) (1964),"Zorro, the Gay Blade (1981)",Zulu (1964),Zus & Zo (2001),[Rec] (2007),eXistenZ (1999),ffolks (a.k.a. North Sea Hijack) (1980),loudQUIETloud: A Film About the Pixies (2006),xXx: State of the Union (2005),"Ãge d'or, L' (1930)"
title,,,,,,,,,,,,,,,,,,,,,
"""Great Performances"" Cats (1998)",1.000000,0.707107,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
'Round Midnight (1986),0.707107,1.000000,0.500000,0.0,0.707107,0.0,0.353553,0.707107,0.0,0.500000,...,0.707107,0.0,0.408248,0.408248,0.0,0.0,0.0,0.0,0.0,0.353553
'Til There Was You (1997),0.000000,0.500000,1.000000,0.0,0.707107,0.0,0.707107,0.707107,0.0,0.500000,...,0.707107,0.0,0.408248,0.816497,0.0,0.0,0.0,0.0,0.0,0.707107
"'burbs, The (1989)",0.000000,0.000000,0.000000,1.0,0.000000,0.5,0.500000,0.000000,1.0,0.000000,...,0.000000,1.0,0.000000,0.577350,0.0,0.0,0.0,0.0,0.0,0.500000
'night Mother (1986),0.000000,0.707107,0.707107,0.0,1.000000,0.0,0.500000,1.000000,0.0,0.707107,...,1.000000,0.0,0.577350,0.577350,0.0,0.0,0.0,0.0,0.0,0.500000
